# Hugging Face Transformers 微调训练入门

本示例将介绍基于 Transformers 实现模型微调训练的主要流程，包括：
- 数据集下载
- 数据预处理
- 训练超参数配置
- 训练评估指标设置
- 训练器基本介绍
- 实战训练
- 模型保存

## YelpReviewFull 数据集

**Hugging Face 数据集：[ YelpReviewFull ](https://huggingface.co/datasets/yelp_review_full)**

### 数据集摘要

Yelp评论数据集包括来自Yelp的评论。它是从Yelp Dataset Challenge 2015数据中提取的。

### 支持的任务和排行榜
文本分类、情感分类：该数据集主要用于文本分类：给定文本，预测情感。

### 语言
这些评论主要以英语编写。

### 数据集结构

#### 数据实例
一个典型的数据点包括文本和相应的标签。

来自YelpReviewFull测试集的示例如下：

```json
{
    'label': 0,
    'text': 'I got \'new\' tires from them and within two weeks got a flat. I took my car to a local mechanic to see if i could get the hole patched, but they said the reason I had a flat was because the previous patch had blown - WAIT, WHAT? I just got the tire and never needed to have it patched? This was supposed to be a new tire. \\nI took the tire over to Flynn\'s and they told me that someone punctured my tire, then tried to patch it. So there are resentful tire slashers? I find that very unlikely. After arguing with the guy and telling him that his logic was far fetched he said he\'d give me a new tire \\"this time\\". \\nI will never go back to Flynn\'s b/c of the way this guy treated me and the simple fact that they gave me a used tire!'
}
```

#### 数据字段

- 'text': 评论文本使用双引号（"）转义，任何内部双引号都通过2个双引号（""）转义。换行符使用反斜杠后跟一个 "n" 字符转义，即 "\n"。
- 'label': 对应于评论的分数（介于1和5之间）。

#### 数据拆分

Yelp评论完整星级数据集是通过随机选取每个1到5星评论的130,000个训练样本和10,000个测试样本构建的。总共有650,000个训练样本和50,000个测试样本。

## 下载数据集

In [2]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")




Generating train split: 100%|████████████████████████████████████████| 650000/650000 [00:03<00:00, 163194.37 examples/s]

Generating test split: 100%|███████████████████████████████████████████| 50000/50000 [00:00<00:00, 216340.41 examples/s]


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [6]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [7]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [8]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(dataset["train"])

,label,text
0,1 star,"Food is crap as well as the mimosa's they offer in the Brunch. They should have more variety. By the way, where is the milk, coffee, croissants, etc? At least I was hoping to find some of this since I went to the Brunch.\n\nI had an omelet and a few sips of the mimosa before run away from this place. It says a lot about how I liked this buffet.\n\nFor the price you pay you should not have high expectations anyway. The next time I hope to remind to check the reviews beforehand."
1,3 stars,"Update, December 2014: I had a total of (I think) 9 sessions on my bikini area and 6 or 7 on my armpits. My last session was at the beginning of September 2014. I began seeing regrowth on my armpits almost immediately, and in November, I began seeing extensive regrowth on my bikini area. This is regrowth of dark hairs that were supposedly eliminated. I have very pale skin and dark hair and am an ideal candidate for successful hair removal; for my final 2-3 sessions, Kristen used a very high setting on the machine to ensure that the hairs were destroyed. \n\nLaser for Less is very clear that laser treatments offer \""permanent hair reduction\"", and Kristen explained to me that some clients have occasional, isolated hair regrowth. I am experiencing a LOT of regrowth, though. Based on this I have to say that laser treatments offer temporary hair reduction, and I am very disappointed. I've lowered my rating from 5 stars to 3 based on the enormous regrowth I am experiencing, and will edit this review again in the future if the regrowth increases.\n\nI've been going to Laser for Less for 5-6 months now, having regular treatments. I started going for a bikini treatment, and the results were so impressive that I decided to add armpits. \n\nKristen is the technician who regularly does my treatments. She is always friendly, but always respectful of my body and space. And she does a great job! From the very first treatment, I could see results. I'm so impressed and happy with Kristen and Laser for Less! I wholeheartedly recommend this place."
2,2 star,"I'm not saying the food isn't good, but overall I feel the experience wasn't so great. It was kind of expensive for what you get and my server was really pushy and tried to recommend pizzas and drinks after we'd made up our mind and ordered. I enjoy recommendations, but when I say I want the Avocado pizza, I want the Avocado pizza and am not interested in the corn one."
3,4 stars,"Great upscale selection. Desserts, pasta, fish etc. great coffee."
4,4 stars,"I am not a buffet connoisseur by any means. I did enjoy this buffet. The food was good with a good selection. I had the champagne breakfast. Wasn't in the mood to drink after the night before, but because I was forced to pay for it, I had a couple of bubbly champagnes."
5,3 stars,"This particular buffet is certainly not the best in town by far, but the food is hot, simple, well prepared, a good amount of choices and the service is good and you might laugh but I like bread pudding and most places it's a bit thick/dense and dry (even with some sauce/glaze on top) but they have the BEST bread pudding I have had at any buffet. So they definitely deserve kudos for that. \nI tried several choices and nothing was bad and I had a very nice simple dinner at a decent buffet."
6,5 stars,"Serving Pure Wax-cellence! All about perfect shaping, waxing, plucking, tinting... Without looking like a drag queen. Stacey is professional, precise, and provides incomparable customer service! Highly recommend The Wax Genie for all your waxing and manscaping needs."
7,2 star,"Classes are good but everyone who works there is rude. I feel uncomfortable every time I walk in there. No acknowledgement whatsoever at the front desk. The yoga teacher and receptionist are busy chatting away about what they are doing later that night. Won't even look at me as I am standing there waiting to pay. Nice facility and decent classes, but terrible customer service."
8,1 s

## 预处理数据

下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。

Datasets 的 `map` 方法，支持一次性在整个数据集上应用预处理函数。

下面使用填充到最大长度的策略，处理整个数据集：

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

Map: 100%|█████████████████████████████████████████████████████████████| 650000/650000 [04:19<00:00, 2501.61 examples/s]

Map: 100%|███████████████████████████████████████████████████████████████| 50000/50000 [00:22<00:00, 2232.57 examples/s]


In [11]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,3 stars,"This place is huge and the stadium seating seems like a great place to watch a game. They also seem to have enough TVs that most tables could see the game of their choice. \n\nAs for the food, it was good but not amazing. The pulled pork sandwich was above average but nothing special, it came with a few thin sliced fried pickles which are so good they should be offered as an appetizer. I was surprised the fries seemed to be the generic frozen fries served at every chain restaurant, they should think about offering them with the fried pickle spices or maybe using hand cut ones.\n\nNot a bad lunch but nothing special unless you are there to watch a game.","[101, 1188, 1282, 1110, 3321, 1105, 1103, 4706, 11051, 3093, 1176, 170, 1632, 1282, 1106, 2824, 170, 1342, 119, 1220, 1145, 3166, 1106, 1138, 1536, 1794, 1116, 1115, 1211, 7072, 1180, 1267, 1103, 1342, 1104, 1147, 3026, 119, 165, 183, 165, 183, 23390, 1111, 1103, 2094, 117, 1122, 1108, 1363, 1133, 1136, 6929, 119, 1109, 1865, 19915, 14327, 1108, 1807, 1903, 1133, 1720, 1957, 117, 1122, 1338, 1114, 170, 1374, 4240, 17053, 15688, 3368, 2897, 1134, 1132, 1177, 1363, 1152, 1431, 1129, 2356, 1112, 1126, 12647, 26883, 6198, 119, 146, 1108, 3753, 1103, 175, 3377, 1882, 1106, 1129, 1103, 13179, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


### 数据抽样

使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer）

`shuffle()`函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [12]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [13]:
small_train_dataset[2]

{'label': 4,
 'text': 'I LOVE Bloom Salon... all of their stylist are very qualified and provide excellent hair care...I prefer to book my appointments with Andrea, but if she is not available I am not afraid to book with anyone else.  Not only does this salon provide hair care, but they also offer skin, nails and massage therapy!!  What a great place with a relaxing atmosphere...I HIGHLY recommend this place.',
 'input_ids': [101,
  146,
  149,
  2346,
  17145,
  19169,
  17420,
  119,
  119,
  119,
  1155,
  1104,
  1147,
  188,
  2340,
  7276,
  1132,
  1304,
  4452,
  1105,
  2194,
  6548,
  1716,
  1920,
  119,
  119,
  119,
  146,
  9353,
  1106,
  1520,
  1139,
  16323,
  1114,
  8326,
  117,
  1133,
  1191,
  1131,
  1110,
  1136,
  1907,
  146,
  1821,
  1136,
  3737,
  1106,
  1520,
  1114,
  2256,
  1950,
  119,
  1753,
  1178,
  1674,
  1142,
  20310,
  2194,
  1716,
  1920,
  117,
  1133,
  1152,
  1145,
  2906,
  2241,
  117,
  10821,
  1105,
  26088,
  7606,
  106,
  106

## 微调训练配置

### 加载 BERT 模型

警告通知我们正在丢弃一些权重（`vocab_transform` 和 `vocab_layer_norm` 层），并随机初始化其他一些权重（`pre_classifier` 和 `classifier` 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 训练超参数（TrainingArguments）

完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L161

**最重要配置：模型权重保存路径(output_dir)**

In [15]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [16]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better=None,
group_by_le

### 训练过程中的指标评估（Evaluate)

**[Hugging Face Evaluate 库](https://huggingface.co/docs/evaluate/index)** 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 **完整评估指标：https://huggingface.co/evaluate-metric**

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标。 

Evaluate库提供了一个简单的准确率函数，您可以使用`evaluate.load`函数加载

In [21]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy.py")


接着，调用 `compute` 函数来计算预测的准确率。

在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（**所有Transformers 模型都返回 logits**）。

In [22]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### 训练过程指标监控

通常，为了监控训练过程中的评估指标变化，我们可以在`TrainingArguments`指定`evaluation_strategy`参数，以便在 epoch 结束时报告评估指标。

In [23]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

## 开始训练

### 实例化训练器（Trainer）

`kernel version` 版本问题：暂不影响本示例代码运行

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

## 使用 nvidia-smi 查看 GPU 使用

为了实时查看GPU使用情况，可以使用 `watch` 指令实现轮询：`watch -n 1 nvidia-smi`:

```shell
Every 1.0s: nvidia-smi                                                   Wed Dec 20 14:37:41 2023

Wed Dec 20 14:37:41 2023
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:0D.0 Off |                    0 |
| N/A   64C    P0              69W /  70W |   6665MiB / 15360MiB |     98%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+

+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A     18395      C   /root/miniconda3/bin/python                6660MiB |
+---------------------------------------------------------------------------------------+
```

In [ ]:
trainer.train()

In [18]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

In [19]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.0753791332244873,
 'eval_accuracy': 0.52,
 'eval_runtime': 2.9889,
 'eval_samples_per_second': 33.457,
 'eval_steps_per_second': 4.349,
 'epoch': 3.0}

### 保存模型和训练状态

- 使用 `trainer.save_model` 方法保存模型，后续可以通过 from_pretrained() 方法重新加载
- 使用 `trainer.save_state` 方法保存训练状态

In [20]:
trainer.save_model(model_dir)

In [21]:
trainer.save_state()

In [23]:
# trainer.model.save_pretrained("./")

## Homework: 使用完整的 YelpReviewFull 数据集训练，看 Acc 最高能到多少